# Import Libraries

In [1]:
import numpy as np
import pandas as pd
from hmmlearn import hmm
import warnings
from constants import *
import math
import random
import matplotlib.pyplot as plt

# Helper Methods

In [9]:
def generate_random_sample(X, size):
    '''
    Given a list X, 
    generate random samples of given size
    '''
    Z_temp = random.sample(list(X), size)
    
    #Concatenation
    Z = [Z_temp[0]]
    for val in Z_temp[1:]:
        Z = np.concatenate([Z,[val]])
    
    return Z

def convert_values_to_list(list_val):
    '''
    Given a list X = [1 2 3] , 
    return X = [[1],[2],[3]]
    '''
    X = []
    for i in list_val:
        X.append([i])
    return X

def conversion_list_of_list(X, DIMENSION):
    '''
    Given a list X with values in lists,
    X = [[ 3  2  2],[4 8 10]]
    Convert each value to a list
    Return list of lists, array of lengths of each sequence
    X = [[3] [2] [2] [4] [8] [10]]
    ''' 
    X_new = []
    length = len(X)
    for idx, val_list in enumerate(X):
        Y = []
        for val in val_list:
            Y.append([val])
        X_new.append(Y)

    #Concatenation
    Z = X_new[0]
    for val_list in X_new[1:]:
        Z = np.concatenate([Z,val_list])

    # assign array of lengths for HMM
    lengths = [DIMENSION]*length
    
    return Z,lengths

#Calculate likelihood for given sequence according to given HMMs and return HMM
def likelihood_sequence(sequence, HMM_array):
    '''
    Given list of K HMMs and sequence,
    determines likelihood of sequence under all HMM models
    Returns index of HMM which has max likelihood
    ''' 
    scores = []
    length = [len(sequence)]
    for i, HMM in enumerate(HMM_array):
        calculated_score = HMM.score(sequence, length)
        scores.append(calculated_score)
    idx = scores.index(max(scores))
    return idx

def HMM_model_stats(model):
    '''
    Details of HMM model
    ''' 
    print("*************************************")
    print("Transition matrix")
    print(model.transmat_)
    print("*************************************")
    print("Means and stds of each hidden state")
    for i in range(model.n_components):
        print("Hidden state {0}".format(i))
        print("mean = ", model.means_[i])
        print("std = ", [np.sqrt(model.covars_[i])])
        print()

def BIC(HMM,X):
    LogLikelihood = model.score(X)
    num_hidden_states = model.n_components
    # D counts transition matrix, emission matrix, sequences estimated (Z), covariance matrix
    D = (num_hidden_states) + 2*(num_hidden_states**2) + len(X)*DIMENSION 
    BIC = LogLikelihood - (D/2)*np.log(len(X))
    return BIC
    
def BIC_array(HMM_array,X_i):
    BIC_total = 0
    for i in range(len(X_i)):
        model = HMM_array[i]
        X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        LogLikelihood = model.score(X,lengths)
        num_hidden_states = model.n_components
        # D counts transition matrix, emission matrix, sequences estimated (Z), covariance matrix
        D = num_hidden_states + 2*(num_hidden_states**2) + len(X)*DIMENSION 
        BIC = LogLikelihood - (D/2)*np.log(len(X))
        BIC_total+= BIC
    return BIC_total

def likelihood_array(HMM_array,X_i):
    likelihood_total = 0
    for i in range(len(X_i)):
        if(len(X_i[i])>=HMM_array[i].n_mix):
            model = HMM_array[i]
            X, lengths = conversion_list_of_list(X_i[i],DIMENSION)
            LogLikelihood = model.score(X,lengths)
            likelihood = LogLikelihood 
            likelihood_total+= likelihood
    return likelihood_total

def plot_BIC(list_k, BIC_score):
    fig = plt.subplot(111)
    plt.plot(list_k, BIC_score, marker='o')  
    plt.xlabel('Value of K')
    plt.ylabel('Objective')
    plt.title('BIC')
    plt.show() 

# Load Data and Clean

In [3]:
df = pd.read_csv('../Data/merged.txt', sep=",", na_values=['-'])
df = df.dropna()
df = df[['cdReads0','cdReads1','cdReads2','cdRPKM0','cdRPKM1','cdRPKM2']]

# Filter cdReads

In [4]:
df = df[(df['cdReads0'] >= 50) & (df['cdReads1'] >= 50) & (df['cdReads2'] >= 50)]

In [5]:
#Dataset
df_main = df[['cdRPKM0','cdRPKM1','cdRPKM2']]
LENGTH,DIMENSION = df_main.shape
print("Dataset size is",LENGTH)
print("Features are", DIMENSION)
print(df_main.head(5))
X = np.log2(df_main.values)
print("****************************")
print("First 5 log2 values\n",X[:5])

Dataset size is 2072
Features are 3
       cdRPKM0     cdRPKM1     cdRPKM2
6    39.456786   62.505536   84.519655
19   29.836928   22.172994   17.721150
34  188.746942  211.104962  221.175677
57  283.823604  374.683044  285.247754
81   90.967704  125.648541  120.892583
****************************
First 5 log2 values
 [[ 5.30220154  5.96591207  6.40121497]
 [ 4.89902709  4.47073168  4.14740034]
 [ 7.56030946  7.72181668  7.78904893]
 [ 8.14885076  8.54952688  8.15607171]
 [ 6.50728253  6.97325011  6.91758192]]


# Arrays with HMM models for 1<=K<=25

In [6]:
HMM_K_ARRAYS = []
X_i_K_ARRAYS = []

# Check likelihood and do assignments

In [10]:
K_values = range(8,30)

I'm already doing the E-step right now, where I do hard assignment by choosing the sequence for which the probability is the maximum. I just need to generate the list of prior probabilities z_k where \sum(z_k)=1. This can be done by taking all the probabilities generated for each HMM and normalizing them. The i^th element in this list z_k represents the probability of being generated by the i^th model, and I will modify the objective likelihood function to be multiplied with this prior probability. 

In [12]:
for K in K_values:
    HMM_array = []
    X_i = []
    print("**************** K =", K ,"************************")
    for i in range(K):
        X_i.append([])

    # Sequences for initial HMM estimation
    for i in range(LENGTH):
        for j in range(K):
            if(i%K==j):
                X_i[j].append(list(X[i]))
    
    NUM_ITERATIONS = 0
    NUM_CLUSTER_PREV = {}
    NUM_CLUSTER_NOW = {}
                
    for i in range(K):
        model = hmm.GMMHMM(n_components=3,n_mix=3,covariance_type='diag')
        X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
        model.fit(X_temp, lengths)
        HMM_array.append(model)
    
    likelihood_prev = likelihood_array(HMM_array,X_i)
    print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_prev)
    NUM_ITERATIONS+=1
    while (True):
        # Assign all sequences to HMM models

        print("************ Check likelihood of sequence in HMM  *********")
        NUM_CLUSTER_NOW = {}
        for x in X:
            sequence = convert_values_to_list(x)
            hmm_index = likelihood_sequence(sequence, HMM_array)
            X_i[hmm_index].append(list(x))
            if (hmm_index not in NUM_CLUSTER_NOW):
                NUM_CLUSTER_NOW[hmm_index] = 1
            else:
                NUM_CLUSTER_NOW[hmm_index] += 1
        print("************ Checking likelihood done  *********")

        # Re-estimate parameters for new HMMs
        print("************ Re-estimating HMM *********")
        HMM_array_prev = HMM_array
        HMM_array = []
        for i in range(K):
            model = hmm.GMMHMM(n_components=3,n_mix=3,covariance_type='diag')
            if(len(X_i[i])>=model.n_mix):
                X_temp, lengths = conversion_list_of_list(X_i[i], DIMENSION)
                model.fit(X_temp, lengths)
                HMM_array.append(model)
            else:
                HMM_array.append(HMM_array_prev[i])
        print("************ Re-estimation done *********")
        print("Previous assignments",NUM_CLUSTER_PREV)
        print("Current assignments",NUM_CLUSTER_NOW)
        likelihood_curr = likelihood_array(HMM_array,X_i)
        print("Likelihood for iteration",NUM_ITERATIONS,"is",likelihood_curr)
        print("*****************************************")

        # if no reassignments, then break
        if ((NUM_CLUSTER_PREV == NUM_CLUSTER_NOW) or NUM_ITERATIONS>100):
            HMM_K_ARRAYS.append(HMM_array)
            X_i_K_ARRAYS.append(X_i)
            break
        else:
            # initialize empty subsets of data for next iteration
            X_i = []
            for i in range(K):
                X_i.append([])

            NUM_CLUSTER_PREV = NUM_CLUSTER_NOW
            print("Num iterations is:", NUM_ITERATIONS)
            NUM_ITERATIONS += 1
            likelihood_prev = likelihood_curr
    print("**********************************************************\n\n")

**************** K = 8 ************************
Likelihood for iteration 0 is -363922.945482
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {}
Current assignments {5: 420, 2: 303, 7: 313, 6: 311, 4: 89, 1: 197, 3: 423, 0: 16}
Likelihood for iteration 1 is -387587.839509
*****************************************
Num iterations is: 1
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {5: 420, 2: 303, 7: 313, 6: 311, 4: 89, 1: 197, 3: 423, 0: 16}
Current assignments {5: 419, 2: 297, 7: 301, 6: 318, 4: 93, 1: 197, 3: 431, 0: 16}
Likelihood for iteration 2 is -9688.7249827
*****************************************
Num iterations is: 2
************ Check likelihood of s

/usr/local/lib/python3.6/site-packages/hmmlearn/hmm.py:929: RuntimeWarning: divide by zero encountered in true_divide
  new_cov = new_cov_numer / new_cov_denom


************ Re-estimation done *********
Previous assignments {8: 293, 5: 266, 1: 299, 0: 359, 7: 84, 4: 201, 2: 540, 6: 29, 3: 1}
Current assignments {8: 266, 5: 267, 1: 300, 0: 367, 4: 201, 2: 564, 7: 81, 6: 25, 3: 1}
Likelihood for iteration 6 is -9821.58925565
*****************************************
Num iterations is: 6
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********
Previous assignments {8: 266, 5: 267, 1: 300, 0: 367, 4: 201, 2: 564, 7: 81, 6: 25, 3: 1}
Current assignments {8: 229, 5: 249, 1: 300, 0: 369, 4: 179, 2: 522, 7: 81, 6: 142, 3: 1}
Likelihood for iteration 7 is -13132.1423571
*****************************************
Num iterations is: 7
************ Check likelihood of sequence in HMM  *********
************ Checking likelihood done  *********
************ Re-estimating HMM *********
************ Re-estimation done *********

ValueError: n_samples=1 should be >= n_clusters=3

In [ ]:
import dill
dill.dump_session('correct-HMM-X_i.db')

# Calculate BIC for these assignments

In [ ]:
print(BIC_array(HMM_array,X_i))

In [ ]:
print(BIC_array(HMM_array,X_i))

# Noise Cluster

In [ ]:
# Get list of lists to fit
Z, lengths = conversion_list_of_list(X,DIMENSION)

In [ ]:
# Noise cluster 

model_noise = hmm.GMMHMM(n_components=3,n_mix=3,covariance_type='spherical')
model_noise.fit(Z, lengths)

In [ ]:
HMM_model_stats(model_noise)

# Generate subsets of data for K HMMs

In [ ]:
K = 3
HMM_array = []
X_i = []

for i in range(K):
    X_i.append([])

# Sequences for initial HMM estimation
for i in range(LENGTH):
    for j in range(K):
        if(i%K==j):
            X_i[j].append(list(X[i]))

In [ ]:
for i in range(K):
    model = hmm.GMMHMM(n_components=3,n_mix=3, covariance_type='diag')
    X_temp, lengths = conversion_list_of_list(X_i[i],DIMENSION)
    model.fit(X_temp, lengths)
    HMM_array.append(model)

## Statistics for K HMMs

In [ ]:
for i in range(K):
    print("Statistics for HMM Model", i)
    HMM_model_stats(HMM_array[i])
    print('\n\n')

In [ ]:
NUM_ITERATIONS = 1
NUM_CLUSTER_PREV = {}
NUM_CLUSTER_NOW = {}

In [ ]:
#initialize empty subsets of data
X_i = []

for i in range(K):
    X_i.append([])

In [ ]:
likelihood_prev = likelihood_array(HMM_array,X_i)
print("Likelihood for iteration",0,"is",likelihood_prev)